In [3]:
import sys
sys.path.append("..")

In [4]:
import torch
from pykeen.models import TransE
from pykeen.triples import TriplesFactory
from llavart.utils.dirutils import get_data_dir, get_model_checkpoints_dir
from llavart.models.graph_retriever.modeling import get_kge_model
from langchain_community.graphs import Neo4jGraph
import os
from tqdm import tqdm
from safetensors.torch import load_file, save_file
import lovely_tensors as lt

lt.monkey_patch()

In [8]:
tf = TriplesFactory.from_path(get_data_dir() / "graph" / "graph.tsv")
training, testing, validation = tf.split([.8, .1, .1], random_state=42)
model = get_kge_model("TransE", training, embs_init_path=get_model_checkpoints_dir() / "graph_retriever" / "init" / "TransE.safetensors", model_kwargs={"embedding_dim": 768})
model

No random seed is specified. This may lead to non-reproducible results.


TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(117147, 768)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(17, 768)
    )
  )
  (weight_regularizers): ModuleList()
)

In [ ]:
graph = Neo4jGraph()
graph

In [ ]:
query = """
MATCH (a: Artwork)
RETURN ID(a) as id, a.name as name
ORDER BY id
"""
result = graph.query(query)
artworks_id2name = {r["id"]: r["name"] for r in result}

In [ ]:
artwork_embeddings = load_file(get_data_dir() / "images-hd_features.safetensors")["features"]

In [ ]:
keys = list(artworks_id2name.keys())
keys = [str(k) for k in keys]

In [ ]:
indexes = []
og_indexes = []
for i, k in enumerate(keys):
    try:
        indexes.append(tf.entities_to_ids([k])[0])
        og_indexes.append(i)
    except:
        pass

In [ ]:
embeddings = model.entity_representations[0]._embeddings

In [ ]:
# set the embeddings
embs = embeddings.weight.detach()
embs

In [ ]:
artwork_embeddings[og_indexes]

In [ ]:
embs[indexes] = artwork_embeddings[og_indexes]
embs

In [ ]:
model.state_dict()

In [ ]:
# get class name of model TransE
save_file({"entity_representations.0._embeddings.weight": embs}, get_model_checkpoints_dir() / "graph_retriever" / "init" / f"{model.__class__.__name__}.safetensors")

In [ ]:
model.load_state_dict(load_file(get_model_checkpoints_dir() / "graph_retriever" / "init" / f"{model.__class__.__name__}.safetensors"), strict=False)